In [ ]:
from pathlib import Path
import sys
sys.path.append(str(Path("..").resolve()))

import pandas as pd
import matplotlib.pyplot as plt

from src.data.loader import fetch_ohlc
from src.strategies.sma_crossover import sma_signal
from src.backtest.engine import run_long_flat
from src.backtest.metrics import (
    summarize, sharpe, sortino, cagr, max_drawdown, equity_curve
)

TICKER = "AAPL"
START = "2015-01-01"
END   = "2025-09-01"
FAST, SLOW = 20, 50
COST_BPS = 1.0
AUTO_ADJUST = True


In [ ]:
close = fetch_ohlc(TICKER, START, END, auto_adjust=AUTO_ADJUST)
signal = sma_signal(close, FAST, SLOW)
res = run_long_flat(close, signal, cost_bps=COST_BPS)

spy = fetch_ohlc("SPY", START, END, auto_adjust=AUTO_ADJUST)
bench_ret = spy.pct_change().reindex(close.index).fillna(0.0)
bench_eq  = equity_curve(bench_ret)

df = pd.DataFrame({
    "close": close.squeeze(),
    "signal": signal.squeeze(),
    "strat_ret": res["returns"].squeeze(),
    "strat_eq": res["equity"].squeeze(),
    "bench_ret": bench_ret.squeeze(),
    "bench_eq": bench_eq.squeeze(),
})
df.tail()


In [ ]:
strat = summarize(df["strat_ret"], df["strat_eq"], df["signal"].shift(1).fillna(0.0))
bench = {
    "Sharpe":  sharpe(bench_ret),
    "Sortino": sortino(bench_ret),
    "CAGR":    cagr(bench_ret),
    "MaxDD":   max_drawdown(bench_eq),
}
pd.DataFrame([strat, bench], index=["Strategy", "SPY"]).T


In [ ]:
# price + SMAs
fast_sma = close.rolling(FAST, min_periods=FAST).mean()
slow_sma = close.rolling(SLOW, min_periods=SLOW).mean()
plt.figure(figsize=(10,4)); close.plot(label=TICKER)
fast_sma.plot(label=f"SMA{FAST}"); slow_sma.plot(label=f"SMA{SLOW}")
plt.title(f"{TICKER} price & SMAs"); plt.legend(); plt.tight_layout(); plt.show()

# equity vs SPY
plt.figure(figsize=(10,4))
df["strat_eq"].plot(label="Strategy"); df["bench_eq"].plot(label="SPY")
plt.title("Equity curves"); plt.legend(); plt.tight_layout(); plt.show()

# drawdown
peak = df["strat_eq"].cummax(); dd = df["strat_eq"]/peak - 1.0
plt.figure(figsize=(10,2.8)); dd.plot(); plt.title("Strategy drawdown")
plt.tight_layout(); plt.show()

# Optional saves:
# df.to_csv(f"assets/{TICKER}_results.csv")
# (pd.Series(strat)).to_json(f"assets/{TICKER}_summary.json", indent=2)
